# data preparation

## BML

In [1]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# !apt-get install unrar

# !unrar x "/content/drive/My Drive/Copy of Phase3.rar" "/content/drive/My Drive/aicv/"

Mounted at /content/drive


In [2]:
pat_list = os.listdir('/content/drive/My Drive/aicv/Phase3')
pat_list = sorted(pat_list)
pat_count = len(pat_list)
print(pat_count)

102


In [ ]:
import shutil

def split(src, dest):
    train_f = os.path.join(dest, "train")
    val_f = os.path.join(dest, "validate")
    test_f = os.path.join(dest, "test")

    if os.path.exists(train_f):
        pass
    else:
        os.mkdir(train_f)

    if os.path.exists(val_f):
        pass
    else:
        os.mkdir(val_f)


    if os.path.exists(test_f):
        pass
    else:
        os.mkdir(test_f)

    pat_list = os.listdir(src)
    pat_list = sorted(pat_list)
    pat_count = len(pat_list)

    pat_list_has_bml_count = 0
    pat_list_has_bml = []
    for pat in pat_list:
      if os.path.exists(os.path.join(src, pat, 'v00/meta/bml_masks')):
        pat_list_has_bml_count += 1
        pat_list_has_bml.append(pat)
    print(len( pat_list), pat_count)
    print(len(pat_list_has_bml), pat_list_has_bml_count)


    train_len = int(pat_list_has_bml_count * 0.7) # 70  0-70 / 18
    val_len = int(pat_list_has_bml_count * 0.15) # 15   71-85 / 3

    pat_index = 0
    for pat in pat_list_has_bml:
      mask_src_folder = os.path.join(src, pat, 'v00/meta/bml_masks')
      bone_src_folder = os.path.join(src, pat, 'v00/meta/images')
      if not os.path.exists(mask_src_folder):
        continue

      target = test_f
      print(pat_index, pat)
      if pat_index < train_len:
          target = train_f
      elif pat_index < train_len + val_len:
          target = val_f

      for fname in os.listdir(mask_src_folder):
          # copying the files to the
          # destination directory
          shutil.copy2(os.path.join(mask_src_folder, fname), target)
          # fanme -->9048789_v00_4_mask.bmp
          # strop--->9048789_v00_4.bmp
          strip_file = fname.replace("_mask.bmp", ".bmp")
          shutil.copy2(os.path.join(bone_src_folder, strip_file), target)


      pat_index += 1

print("start")
split(r'/content/drive/My Drive/aicv/Phase3', r'/content/drive/My Drive/aicv/bml_data_split')
print("end")


## dataset

In [2]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset

In [3]:
from easydict import EasyDict

cfg = EasyDict()
cfg.batch_size = 4
cfg.epoch = 1000
cfg.learning_rate = 1e-4
cfg.momentum = 0.9
cfg.weight_decay = 1e-4
cfg.patience = 100
cfg.inference_threshold = 0.75

cfg.transunet = EasyDict()
cfg.transunet.img_dim = 448
cfg.transunet.in_channels = 3
cfg.transunet.out_channels = 128
cfg.transunet.head_num = 4
cfg.transunet.mlp_dim = 448
cfg.transunet.block_num = 8
cfg.transunet.patch_dim = 16
cfg.transunet.class_num = 1

In [4]:
class DentalDataset(Dataset):
    output_size = cfg.transunet.img_dim

    def __init__(self, path, transform):
        super().__init__()

        self.transform = transform
        self.img_paths = []
        self.mask_paths = []
        for p in os.listdir(path):
            name_array = p.split('.')[0].split('_')
            if len(name_array) == 4 : #mask
              self.mask_paths.append(os.path.join(path, p))
            else:
              self.img_paths.append(os.path.join(path, p))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img = self.img_paths[idx]
        mask = self.mask_paths[idx]

        img = cv2.imread(img)
        #print(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.output_size, self.output_size))

        mask = cv2.imread(mask, 0)
        mask = cv2.resize(mask, (self.output_size, self.output_size), interpolation=cv2.INTER_NEAREST)
        mask = np.expand_dims(mask, axis=-1)

        sample = {'img': img, 'mask': mask}

        if self.transform:
            sample = self.transform(sample)

        img, mask = sample['img'], sample['mask']

        img = img / 255.
        img = img.transpose((2, 0, 1))
        img = torch.from_numpy(img.astype('float32'))

        mask = mask / 255.
        mask = mask.transpose((2, 0, 1))
        mask = torch.from_numpy(mask.astype('float32'))

        return {'img': img, 'mask': mask}

    def __len__(self):
        return len(self.img_paths)

In [5]:
md = DentalDataset('/content/drive/My Drive/aicv/bml_data_split/train',
                    None)

for sample in md:
    print(sample['img'].shape)
    print(sample['mask'].shape)

    break

torch.Size([3, 448, 448])
torch.Size([1, 448, 448])


# train

In [6]:
from tqdm import tqdm
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

# Additional Scripts
import sys
sys.path.append('/content/drive/My Drive/aicv/bml_code')

In [8]:
! pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00


In [9]:
from torch.optim import SGD

# Additional Scripts
from transunet import TransUNet
from utils import dice_loss

class TransUNetSeg:
    def __init__(self, device):
        self.device = device
        self.model = TransUNet(img_dim=cfg.transunet.img_dim,
                               in_channels=cfg.transunet.in_channels,
                               out_channels=cfg.transunet.out_channels,
                               head_num=cfg.transunet.head_num,
                               mlp_dim=cfg.transunet.mlp_dim,
                               block_num=cfg.transunet.block_num,
                               patch_dim=cfg.transunet.patch_dim,
                               class_num=cfg.transunet.class_num).to(self.device)

        self.criterion = dice_loss
        self.optimizer = SGD(self.model.parameters(), lr=cfg.learning_rate,
                             momentum=cfg.momentum, weight_decay=cfg.weight_decay)

    def load_model(self, path):
        ckpt = torch.load(path)
        self.model.load_state_dict(ckpt['model_state_dict'])
        self.optimizer.load_state_dict(ckpt['optimizer_state_dict'])

        self.model.eval()

    def train_step(self, **params):
        self.model.train()

        self.optimizer.zero_grad()
        pred_mask = self.model(params['img'])
        loss = self.criterion(pred_mask, params['mask'])
        loss.backward()
        self.optimizer.step()

        return loss.item(), pred_mask

    def test_step(self, **params):
        self.model.eval()

        pred_mask = self.model(params['img'])
        loss = self.criterion(pred_mask, params['mask'])

        return loss.item(), pred_mask

In [10]:
from utils import EpochCallback
import transforms as T

class TrainTestPipe:
    def __init__(self, train_path, test_path, model_path, device):
        self.device = device
        self.model_path = model_path

        self.train_loader = self.__load_dataset(train_path, train=True)
        self.test_loader = self.__load_dataset(test_path)

        self.transunet = TransUNetSeg(self.device)

    def __load_dataset(self, path, train=False):
        shuffle = False
        transform = False

        if train:
            shuffle = True
            transform = transforms.Compose([T.RandomAugmentation(2)])

        set = DentalDataset(path, transform)
        loader = DataLoader(set, batch_size=cfg.batch_size, shuffle=shuffle)

        return loader

    def __loop(self, loader, step_func, t):
        total_loss = 0

        for step, data in enumerate(loader):
            img, mask = data['img'], data['mask']
            img = img.to(self.device)
            mask = mask.to(self.device)

            loss, cls_pred = step_func(img=img, mask=mask)

            total_loss += loss

            t.update()

        return total_loss

    def train(self):
        callback = EpochCallback(self.model_path, cfg.epoch,
                                 self.transunet.model, self.transunet.optimizer, 'test_loss', cfg.patience)

        for epoch in range(cfg.epoch):
            with tqdm(total=len(self.train_loader) + len(self.test_loader)) as t:
                train_loss = self.__loop(self.train_loader, self.transunet.train_step, t)

                test_loss = self.__loop(self.test_loader, self.transunet.test_step, t)

            callback.epoch_end(epoch + 1,
                               {'loss': train_loss / len(self.train_loader),
                                'test_loss': test_loss / len(self.test_loader)})

            if callback.end_training:
                break

In [17]:
import sys
import torch
import argparse

# Additional Scripts
from inference import SegInference

import sys

sys.argv = ['placeholder', '--mode', 'train',
            '--model_path', '/content/drive/My Drive/aicv/bml_code/model1.pth',
            '--train_path', '/content/drive/My Drive/aicv/bml_data_split/train',
            '--test_path', '/content/drive/My Drive/aicv/bml_data_split/test']



def main_pipeline(parser):
    device = 'cpu:0'
    if torch.cuda.is_available():
        device = 'cuda:0'

    if parser.mode == 'train':
        ttp = TrainTestPipe(train_path=parser.train_path,
                            test_path=parser.test_path,
                            model_path=parser.model_path,
                            device=device)

        ttp.train()

    elif parser.mode == 'inference':
        inf = SegInference(model_path=parser.model_path,
                           device=device)

        _ = inf.infer(parser.image_path)


if __name__ == '__main__':

  parser = argparse.ArgumentParser()
  parser.add_argument('--mode', type=str, required=True, choices=['train', 'inference'])
  parser.add_argument('--model_path', required=True, type=str, default=None)

  parser.add_argument('--train_path', required='train' in sys.argv,  type=str, default=None)
  parser.add_argument('--test_path', required='train' in sys.argv, type=str, default=None)

  parser.add_argument('--image_path', required='infer' in sys.argv, type=str, default=None)
  args = parser.parse_args()

  main_pipeline(args)


100%|██████████| 101/101 [00:25<00:00,  4.01it/s]


Epoch 1/1000 - loss: 0.9792 test_loss: 0.9894 
test_loss decreased from inf to 0.9894
Model saved to /content/drive/My Drive/aicv/bml_code/model1.pth


100%|██████████| 101/101 [00:25<00:00,  3.98it/s]


Epoch 2/1000 - loss: 0.9785 test_loss: 0.9887 
test_loss decreased from 0.9894 to 0.9887
Model saved to /content/drive/My Drive/aicv/bml_code/model1.pth


100%|██████████| 101/101 [00:25<00:00,  3.94it/s]


...

Epoch 52/1000 - loss: 0.9657 test_loss: 0.9804 
test_loss did not decrease from 0.9804, model did not save!


100%|██████████| 101/101 [00:25<00:00,  4.01it/s]


Epoch 53/1000 - loss: 0.9652 test_loss: 0.9797 
test_loss decreased from 0.9804 to 0.9797
Model saved to /content/drive/My Drive/aicv/bml_code/model1.pth


  1%|          | 1/101 [00:00<01:06,  1.50it/s]


KeyboardInterrupt: ignored